### Import the needed packages

In [175]:
from dotenv import load_dotenv
import os
import snowflake.connector
from snowflake.connector.errors import ProgrammingError

load_dotenv()

True

**Create the connection**

In [176]:
conn = snowflake.connector.connect(
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
    database=os.getenv('SNOWFLAKE_DATABASE'),
    schema=os.getenv('SNOWFLAKE_SCHEMA'),
    # private_key=os.getenv('SNOWFLAKE_PRIVATE_KEY'),
    session_parameters={
        'QUERY_TAG': 'snow-test',
    }
)

**Creating a Database, Schema, and Warehouse** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#creating-a-database-schema-and-warehouse)

In [177]:
conn.cursor().execute("ALTER SESSION SET QUERY_TAG = 'snow-test'")

conn.cursor().execute("CREATE WAREHOUSE IF NOT EXISTS tiny_warehouse_mg")
conn.cursor().execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
conn.cursor().execute("USE DATABASE testdb_mg")
conn.cursor().execute("CREATE SCHEMA IF NOT EXISTS testschema_mg")

**Using the Database, Schema, and Warehouse** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#using-the-database-schema-and-warehouse)

In [178]:
conn.cursor().execute("USE WAREHOUSE tiny_warehouse_mg")
conn.cursor().execute("USE DATABASE testdb_mg")
conn.cursor().execute("USE SCHEMA testdb_mg.testschema_mg")

**Creating Tables and Inserting Data** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#creating-tables-and-inserting-data)


In [179]:
conn.cursor().execute(
    "CREATE OR REPLACE TABLE "
    "test_table(col1 integer, col2 string)")

conn.cursor().execute(
    "INSERT INTO test_table(col1, col2) VALUES " + 
    "    (123, 'test string1'), " + 
    "    (456, 'test string2')")

**Loading data** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#loading-data)


Instead of inserting data into tables using individual INSERT commands, 
you can bulk load data from files staged in either an internal or external location.


**Copying Data from an Internal Location**

In [180]:
try:
    conn.cursor().execute(
        "CREATE OR REPLACE TABLE "
        "mock_data(id integer, first_name string, last_name string, email string, gender string, ip_address string)")

    conn.cursor().execute("PUT file:///home/softsuave/project/snowflake/tmp/data/MOCK_DATA.csv @%mock_data")
    conn.cursor().execute("COPY INTO mock_data")
except Exception as ex:
    print(ex)

**Copying Data from an External Location** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#copying-data-from-an-external-location)

In [181]:
try:
    # Copying Data
    conn.cursor().execute("""
    COPY INTO testtable FROM s3://<s3_bucket>/data/
        STORAGE_INTEGRATION = myint
        FILE_FORMAT=(field_delimiter=',')
    """.format(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY))
except Exception as ex:
    print(ex)

name 'AWS_ACCESS_KEY_ID' is not defined


**Querying Data** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#querying-data)


With the Snowflake Connector for Python, you can submit:

    1.a synchronous query, which returns control to your application after the query completes.

    2.an asynchronous query, which returns control to your application before the query completes.

After the query has completed, you use the Cursor object to fetch the values in the results. 
By default, the Snowflake Connector for Python converts the values from Snowflake data types to native 
Python data types. (Note that you can choose to return the values as strings and perform the type 
conversions in your application. See Improving Query Performance by Bypassing Data Conversion.


**Performing a Synchronous Query**

To perform a synchronous query, call the `execute()` method in the Cursor object. For example:

In [182]:
cur = conn.cursor()
try:
    # we are already mentioned what database, warehouse and schema have to use
    print(cur.execute('select * from mock_data'))
except Exception as ex:
    print(ex)


**Performing an `Asynchronous` Query**


In [183]:
async_query = cur.execute_async('''ALTER TABLE mock_data
                            ADD date_of_birth DATE ;''')
async_query

{'queryId': '01a94ee4-0000-df9b-0000-0000d31155cd',
 'getResultUrl': '/queries/01a94ee4-0000-df9b-0000-0000d31155cd/result',
 'queryAbortsAfterSecs': 300,
 'progressDesc': None}

Checking the Status of a Query

In [184]:
if conn.get_query_status(async_query.get('queryId')).name == 'SUCCESS':
    print('query exceqution is finished')
else:
    print('Query exceqution is not finised')

query exceqution is finished


**To retrieve the results of the query**

Get the results from a query.

In [185]:
async_query = cur.execute_async('''select * from mock_data;''')
cur.get_results_from_sfqid(async_query.get('queryId'))
results = cur.fetchall()
print(f'{results[0]}')

(1, 'Lewie', 'Connah', 'lconnah0@jugem.jp', 'Male', '10.35.171.161', None)


**Retrieving the Snowflake Query ID** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#retrieving-the-snowflake-query-id)

A query ID identifies each query executed by Snowflake. When you use the Snowflake Connector for Python to execute a query, you can access the query ID through the sfqid attribute in the Cursor object:

In [186]:
# Retrieving a Snowflake Query ID
# cur = conn.cursor()
cur.execute("SELECT * FROM mock_data")
print(cur.sfqid)

01a94ee4-0000-df91-0000-0000d3114619


**Using cursor to Fetch Values** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#using-cursor-to-fetch-values)

Fetch values from a table using the cursor object iterator method.

For example, to fetch columns named `first_name` and `last_name` from the table named testtable, which was created earlier (in Creating Tables and Inserting Data), use code similar to the following:

In [187]:
cur = conn.cursor()
try:
    cur.execute("SELECT first_name, last_name FROM mock_data ORDER BY first_name")
    for (first_name, last_name) in cur:
        print('{0}, {1}'.format(first_name, last_name))
except Exception as ex:
    print(ex)
# finally:
    # cur.close()

Aarika, Rubke
Abbe, McKenny
Abbie, Vasser
Abbye, Probet
Abie, Le Cornu
Abigale, Sydes
Adah, Prover
Adan, Savory
Adrian, Ramberg
Aggi, Tebboth
Ainslee, Melloi
Alameda, Eam
Alanah, Starzaker
Aldin, Marmion
Aldin, Clyma
Aldus, Flann
Aldwin, Burrells
Aleda, Grimme
Alexi, Bawden
Alexine, Carnegy
Alfie, Silverstone
Alford, Newbegin
Alfred, Bartolomeotti
Ali, Errington
Alick, Haps
Alla, Tiernan
Allie, Elmore
Allie, Kinchlea
Alma, Kevane
Aloysia, Fuggles
Alphard, Deschelle
Alphard, Egdal
Alvan, Kew
Alvy, Rentalll
Alyosha, Martinetto
Alyosha, Brunt
Alysia, Giorgielli
Amalita, Dunan
Amata, Sheffield
Ambrosi, Cristoforetti
Anatol, Verson
Ancell, Bott
Andreas, Keith
Andriana, Thiese
Angelina, MacCahey
Anna-diane, Noads
Annadiane, Kowalik
Anne, Adlem
Annice, Broggetti
Antonietta, Ceci
Arabelle, Wilcocke
Archambault, Kupec
Ardith, Cade
Aretha, Fairholme
Aridatha, Le Barr
Arie, Fargher
Aristotle, Mitchenson
Arlen, Levene
Arlene, Shillinglaw
Arliene, Windless
Arlina, Simic
Arly, Quincey
Armin, Laurisc

Alternatively, the Snowflake Connector for Python provides a convenient shortcut:

In [188]:
# for (col1, col2) in conn.cursor().execute("SELECT first_name, last_name FROM mock_data"):
#     print('{0}, {1}'.format(col1, col2))

# If you need to get a single result (i.e. a single row), use the fetchone method:

col1, col2 = conn.cursor().execute("SELECT first_name, last_name FROM mock_data").fetchone()
print('{0}, {1}'.format(col1, col2))

# If you need to get the specified number of rows at a time, use the fetchmany method with the number of rows:
cur = conn.cursor().execute("SELECT first_name, last_name FROM mock_data")
ret = cur.fetchmany(3)
print(ret)
# while len(ret) > 0:
#     ret = cur.fetchmany(3)
#     print(ret)

Lewie, Connah
[('Lewie', 'Connah'), ('Fionnula', 'Itzkin'), ('Pauly', 'Pantlin')]


If you need to get all results at once: `fetchall`

In [189]:
results = conn.cursor().execute("SELECT TOP 5 first_name, last_name FROM mock_data").fetchall()
for rec in results:
    print('%s, %s' % (rec[0], rec[1]))

Lewie, Connah
Fionnula, Itzkin
Pauly, Pantlin
Powell, Hurley
Prentiss, Rodrigues


**Using the Timer in the query**

To set a timeout for a query, execute a `begin` command and include a timeout parameter on the query. If the query exceeds the length of the parameter value, an error is produced and a rollback occurs.

In the following code, error 604 means the query was canceled. The timeout parameter starts `Timer()` and cancels if the query does not finish within the specified time.

In [190]:
conn.cursor().execute("create or replace table testtbl(a int, b string)")

conn.cursor().execute("begin")
try:
   conn.cursor().execute("insert into testtbl(a,b) values(3, 'test3'), (4,'test4')", timeout=10) # long query

except ProgrammingError as e:
   if e.errno == 604:
      print("timeout")
      conn.cursor().execute("rollback")
   else:
      raise e
else:
   conn.cursor().execute("commit")

print('timer')

timer


**Using `DictCursor` to Fetch Values by Column Name**

If you want to fetch a value by column name, create a `cursor` object of type `DictCursor`.



In [191]:
# Querying data by DictCursor
from snowflake.connector import DictCursor
cur = conn.cursor(DictCursor)
try:
    cur.execute("SELECT TOP 5 first_name, last_name FROM mock_data")
    for rec in cur:
        print('{0}, {1}'.format(rec['FIRST_NAME'], rec['LAST_NAME']))
finally:
    cur.close()

Lewie, Connah
Fionnula, Itzkin
Pauly, Pantlin
Powell, Hurley
Prentiss, Rodrigues


**Improving Query Performance by Bypassing Data Conversion** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#improving-query-performance-by-bypassing-data-conversion)

To improve query performance, use the `SnowflakeNoConverterToPython` class in the snowflake.connector.converter_null module to bypass data conversions from the Snowflake internal data type to the native Python data type, e.g.:

In [192]:
from snowflake.connector.converter_null import SnowflakeNoConverterToPython

# con = snowflake.connector.connect(
#     ...
#     converter_class=SnowflakeNoConverterToPython
# )

conn_ = snowflake.connector.connect(
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
    database=os.getenv('SNOWFLAKE_DATABASE'),
    schema=os.getenv('SNOWFLAKE_SCHEMA'),
    # private_key=os.getenv('SNOWFLAKE_PRIVATE_KEY'),

    converter_class=SnowflakeNoConverterToPython,

    session_parameters={
        'QUERY_TAG': 'snow-test',
    }
)


for rec in conn_.cursor().execute("SELECT TOP 5 * FROM testdb_mg.testschema_mg.mock_data"):
    # rec includes raw Snowflake data
    print(rec)

(1, 'Lewie', 'Connah', 'lconnah0@jugem.jp', 'Male', '10.35.171.161', None)
(2, 'Fionnula', 'Itzkin', 'fitzkin1@1und1.de', 'Female', '5.23.72.180', None)
(3, 'Pauly', 'Pantlin', 'ppantlin2@mlb.com', 'Female', '189.217.213.121', None)
(4, 'Powell', 'Hurley', 'phurley3@msu.edu', 'Male', '159.241.127.39', None)
(5, 'Prentiss', 'Rodrigues', 'prodrigues4@jimdo.com', 'Male', '168.204.224.114', None)


**Binding Data** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#binding-data)

To specify values to be used in a SQL statement, you can include literals in the statement, or you can bind variables. When you bind variables, you put one or more placeholders in the text of the SQL statement, and then specify the variable (the value to be used) for each placeholder.

**Literals:**

In [193]:
conn.cursor().execute("INSERT INTO mock_data(id,first_name, last_name, email, ip_address) VALUES(1001,'Antony', 'Kumar', 'antony@gmail.com', '192.3.0.3')")
for rec in conn.cursor().execute("select * from mock_data where id>=1000"):
    print(rec)

(1000, 'Gabriela', 'Gatley', 'ggatleyrr@epa.gov', 'Female', '168.212.228.254', None)
(1001, 'Antony', 'Kumar', 'antony@gmail.com', None, '192.3.0.3', None)


**Binding:**

Snowflake supports the following types of binding:

* `pyformat` and `format`, which bind data on the client.

* `qmark` and `numeric`, which bind data on the server.

In [194]:
conn.cursor().execute(
    "INSERT INTO mock_data(id,first_name, last_name, email, ip_address) "
    "VALUES(%s, %s, %s, %s, %s)", (
        1002,
        'settu',
        'pillai',
        'settu@gmail.com',
        '192.0.3.2'
    ))
for rec in conn.cursor().execute("select * from mock_data where id>=1000"):
    print(rec)

(1000, 'Gabriela', 'Gatley', 'ggatleyrr@epa.gov', 'Female', '168.212.228.254', None)
(1001, 'Antony', 'Kumar', 'antony@gmail.com', None, '192.3.0.3', None)
(1002, 'settu', 'pillai', 'settu@gmail.com', None, '192.0.3.2', None)


**Retrieving Column Metadata** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#retrieving-column-metadata)

To retrieve metadata about each column in the result set (e.g. the name, type, precision, scale, etc. of each column), use one of the following approaches:

* To access the metadata after calling the `execute()` method to execute the query, use the description attribute of the Cursor object.

* To access the metadata without having to execute the query, call the `describe()` method.

The describe method is available in the Snowflake Connector for Python 2.4.6 and more recent versions.

**Example: Getting the column name metadata by index (versions 2.4.5 and earlier):**

The following example uses the `description` attribute to retrieve the list of column names after executing a query. The attribute is a list of ResultMetaData objects, and the example accesses the column name from the name attribute of each ResultMetadata object.

In [195]:
cur = conn.cursor()
cur.execute("SELECT * FROM mock_data")
print(','.join([col[0] for col in cur.description]))
cur.description

ID,FIRST_NAME,LAST_NAME,EMAIL,GENDER,IP_ADDRESS,DATE_OF_BIRTH


[ResultMetadata(name='ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='FIRST_NAME', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='LAST_NAME', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='EMAIL', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='GENDER', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='IP_ADDRESS', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='DATE_OF_BIRTH', type_code=3, display_size=None, internal_size=None, precision=None, scale=None, is_nullable=True)]

**Example: Getting the column name metadata by attribute (versions 2.4.6 and later):**

The following example uses the `describe` method to retrieve the list of column names without executing a query. The `describe()` method returns a list of ResultMetaData objects, and the example accesses the column name from the name attribute of each ResultMetadata object.

In [196]:
cur = conn.cursor()
result_metadata_list = cur.describe("SELECT * FROM mock_data")
print(','.join([col.name for col in result_metadata_list]))

ID,FIRST_NAME,LAST_NAME,EMAIL,GENDER,IP_ADDRESS,DATE_OF_BIRTH


**Handling Errors** [CLICK HERe](https://docs.snowflake.com/en/user-guide/python-connector-example.html#handling-errors)

The application must handle exceptions raised from Snowflake Connector properly and decide to continue or stop running the code.



In [197]:
# Catching the syntax error
cur = conn.cursor()
try:
    cur.execute("SELECT * FROM testtable")
except snowflake.connector.errors.ProgrammingError as e:
    # default error message
    print(e)
    # customer error message
    print('Error {0} ({1}): {2} ({3})'.format(e.errno, e.sqlstate, e.msg, e.sfqid))
finally:
    print('-----CLOSE THE CONNECTION----')
    cur.close()

002003 (42S02): SQL compilation error:
Object 'TESTTABLE' does not exist or not authorized.
Error 2003 (42S02): 002003 (42S02): SQL compilation error:
Object 'TESTTABLE' does not exist or not authorized. (01a94ee4-0000-df9b-0000-0000d31155ed)
-----CLOSE THE CONNECTION----


**Using execute_stream to Execute SQL Scripts**

The `execute_stream` function enables you to run one or more SQL scripts in a stream:

In [198]:
from codecs import open
with open('./sqlfile.sql', 'r', encoding='utf-8') as f:
    for cur in conn.execute_stream(f):
        for ret in cur:
            print(ret)
        print('--------------------------------------------------------------------------------')

(1, 'Lewie', 'Connah', 'lconnah0@jugem.jp', 'Male', '10.35.171.161', None)
(2, 'Fionnula', 'Itzkin', 'fitzkin1@1und1.de', 'Female', '5.23.72.180', None)
--------------------------------------------------------------------------------
(1, 'Lewie', 'Connah', 'lconnah0@jugem.jp', 'Male', '10.35.171.161', None)
(2, 'Fionnula', 'Itzkin', 'fitzkin1@1und1.de', 'Female', '5.23.72.180', None)
(3, 'Pauly', 'Pantlin', 'ppantlin2@mlb.com', 'Female', '189.217.213.121', None)
--------------------------------------------------------------------------------
(1, 'Lewie', 'Connah', 'lconnah0@jugem.jp', 'Male', '10.35.171.161', None)
(2, 'Fionnula', 'Itzkin', 'fitzkin1@1und1.de', 'Female', '5.23.72.180', None)
(3, 'Pauly', 'Pantlin', 'ppantlin2@mlb.com', 'Female', '189.217.213.121', None)
(4, 'Powell', 'Hurley', 'phurley3@msu.edu', 'Male', '159.241.127.39', None)
(5, 'Prentiss', 'Rodrigues', 'prodrigues4@jimdo.com', 'Male', '168.204.224.114', None)
--------------------------------------------------------

**Closing the Connection**

As a best practice, close the connection by calling the `close` method:

In [199]:
conn_.close()  # we created two connections one is 'conn' and 'conn_'

**Logging** [CLICK HERE](https://docs.snowflake.com/en/user-guide/python-connector-example.html#logging)

The Snowflake Connector for Python leverages the standard Python `logging` module to log status at regular intervals so that the application can trace its activity working behind the scenes. The simplest way to enable logging is call `logging.basicConfig()` in the beginning of the application.

In [206]:
# create a folder
import os
try:
    os.makedirs('./snowflake', 'log')
except:
    print('ok')

ok
